# Calibration and generation of the input file for 3C-SiC chemical vapor deposition

## Variables to be set

This notebook allows to set up a calibartion for 3C-SiC and to generate the input file `start.dat` of mulskips code. This calibration is the one employed in the manuscript "G. Fisicaro et al. Appl. Phys. Rev. 7(2) 021402 (2020)".

In [1]:
# Temperature of the epitaxial process [K]
Temp = 1500

# Probability for the stacking fault generation
PtransZig = 1.0

# Set the super lattice KMC box
box = [480,480,960]

# Variable to set for the initial system:
# S Sphere
# C Parellelepipid
# F Flat (100) surface
# A Antiphase boundary APB
# I inverted pyramid
# D inverted pyramid of C
# Z inverted pyramid of Si
# J inverted pyramid with APB
Simulation = "F"

# Size of the initial system
Seed_box = [120, 120, 120] # Len1 Len2 Len3

## Evaporation Frequencies

Generation of the evaporation frequencies.

### Ab-initio energetics

In [2]:
import numpy as np
import math
kb=8.617330350e-5 # [eV/K]
kbJ=1.3806485279e-23 # [J/K]
NA=6.02214085774e23 # [mol^-1]
pi=3.141592653589793

In [3]:
kT=kb*Temp

In [4]:
print(kT)

0.12925995525


In [5]:
ESi=np.zeros((4, 4)) # Si energetics of evaporation in eV
EC=np.zeros((4, 4)) # C energetics of evaporation in eV

In [6]:
f=4 # factor for the energetics interpolation

In [7]:
# Coordination 1 of Si from 111 C-terminated 4x1x2 bigdft surface
ESi[0,1]= -6.83 # -4.39 BigDFT # -6.83 # QE 
# Coordination 2 of Si from 100 Si-terminated 3x1x3 dimer surface
ESi[0,2]=-7.04
# Coordination 3 of Si from 111 Si-terminated 4x1x2 surface
ESi[0,3]=-7.77
# Coordination 1 of Si adatom optimized on 111 Si-terminated 4x1x2 surface
ESi[1,0]=-6.72
ESi[1,1]=(f*ESi[1,0]+ESi[0,2])/(f+1) # -6.8 # ESi[1,0] < ESi[1,1] < ESi[0,2]
ESi[1,2]=(f*ESi[0,2]+ESi[0,3])/(f+1) # -6.8 # ESi[0,2] < ESi[1,2] < ESi[0,3]
ESi[2,1]=(f*ESi[1,1]+ESi[0,3])/(f+1) # -6.8 # ESi[1,1] < ESi[2,1] < ESi[0,3]
ESi[2,0]=(f*ESi[1,0]+ESi[2,1])/(f+1) # -6.8 # ESi[1,0] < ESi[2,0] < ESi[2,1]
ESi[3,0]=(f*ESi[2,0]+ESi[2,1])/(f+1) # -6.8 # ESi[2,0] < ESi[3,0] < ESi[2,1]

In [8]:
print(ESi)

[[ 0.       -6.83     -7.04     -7.77    ]
 [-6.72     -6.784    -7.186     0.      ]
 [-6.77224  -6.9812    0.        0.      ]
 [-6.814032  0.        0.        0.      ]]


In [9]:
# Coordination 1 of C from 111 Si-terminated 4x1x2 surface
EC[1,0]=-9.10
# Coordination 2 of C from 100 C-terminated 3x1x3 dimer surface
EC[2,0]=-11.43
# Coordination 3 of C from 111 C-terminated 4x1x2 bigdft surface
EC[3,0]=-11.80
# Coordination 1 of C adatom optimized on 111 C-terminated 4x1x2 bigdft surface
EC[0,1]=-7.76
EC[1,1]= (f*EC[1,0]+EC[2,0])/(f+1) #  -9.8 # EC[1,0] < EC[1,1] < EC[2,0]
EC[1,2]= (f*EC[1,1]+EC[3,0])/(f+1) # -10.0 # EC[1,1] < EC[1,2] < EC[3,0]
EC[0,2]= (f*EC[0,1]+EC[1,2])/(f+1) # -8.1 # EC[0,1] < EC[0,2] < EC[1,2] 
EC[2,1]= (f*EC[2,0]+EC[3,0])/(f+1) # -11.6 # EC[2,0] < EC[2,1] < EC[3,0]
EC[0,3]= (f*EC[0,2]+EC[1,2])/(f+1) # -9.1 # EC[0,2] < EC[0,3] < EC[1,2]

In [10]:
print(EC)

[[  0.        -7.76      -8.21056   -8.571008]
 [ -9.1       -9.566    -10.0128     0.      ]
 [-11.43     -11.504      0.         0.      ]
 [-11.8        0.         0.         0.      ]]


### Langmuir prefactor

Following we derive prefactors for the evaporation rates starting from the experimental partial pressures of Lilov 1993's work.

In [11]:
A_1={'Si':-27499.8,'Si2C':-34177.2,'SiC2':-34075.8}      # Temperature range 1500-2000 [K]
A_2={'Si':-27261.32,'Si2C':-33500.33,'SiC2':-33526.61}   # Temperature range 2000-2546 [K]
B_1={'Si':12.8114,'Si2C':15.1877,'SiC2':15.4274}         # Temperature range 1500-2000 [K]
B_2={'Si':12.6921,'Si2C':14.8493,'SiC2':15.1528}         # Temperature range 2000-2546 [K]

In [12]:
Pexp={} # [N/m^2]
if 1500 <= Temp <=2000:
    ele='Si'
    Pexp[ele]=math.exp(A_1[ele]/Temp+B_1[ele])
    ele='Si2C'
    Pexp[ele]=math.exp(A_1[ele]/Temp+B_1[ele])
    ele='SiC2'
    Pexp[ele]=math.exp(A_1[ele]/Temp+B_1[ele])
elif 2000 < Temp <=2546:
    ele='Si'
    Pexp[ele]=math.exp(A_2[ele]/Temp+B_2[ele])
    ele='Si2C'
    Pexp[ele]=math.exp(A_2[ele]/Temp+B_2[ele])
    ele='SiC2'
    Pexp[ele]=math.exp(A_2[ele]/Temp+B_2[ele])
else:
    print("error, temperature range not yet implemented.")

In [13]:
print(Pexp)

{'Si': 0.0039986439013764, 'Si2C': 0.0005019048490315947, 'SiC2': 0.0006824645320143978}


In [14]:
mC=12
mSi=28

In [15]:
PSi=Pexp['Si']+(2*mSi/(2*mSi+mC))*Pexp['Si2C']+(mSi/(mSi+2*mC))*Pexp['SiC2'] # [N/m^2]

In [16]:
PC=(mC/(2*mSi+mC))*Pexp['Si2C']+(2*mC/(mSi+2*mC))*Pexp['SiC2'] # [N/m^2]

In [17]:
print(PSi)

0.004779458208315149


In [18]:
print(PC)

0.00040355507410724334


### Langmuir setup for evaporation probabilities

In [19]:
Bolt_Si=np.zeros((4, 4)) # Probabilities of Si
Bolt_C=np.zeros((4, 4)) # Probabilities of C
#shift_Si = 6.5
#shift_C = 11.0

In [20]:
Bolt_Si=np.exp(ESi/kT)
Bolt_C =np.exp(EC/kT)

In [21]:
sum_Si=Bolt_Si[0,2]
sum_C=Bolt_C[2,0]

In [22]:
print(Bolt_Si)

[[1.00000000e+00 1.12772249e-23 2.22143217e-24 7.83321736e-27]
 [2.64110763e-23 1.60973909e-23 7.17950465e-25 1.00000000e+00]
 [1.76306111e-23 3.50100256e-24 1.00000000e+00 1.00000000e+00]
 [1.27600501e-23 1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [23]:
print(Bolt_C)

[[1.00000000e+00 8.46328002e-27 2.59252663e-28 1.59467005e-29]
 [2.66278035e-31 7.23841909e-33 2.28276544e-34 1.00000000e+00]
 [3.95254888e-39 2.22970860e-39 1.00000000e+00 1.00000000e+00]
 [2.25803955e-40 1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [24]:
PvSi=PSi*Bolt_Si/sum_Si # [N/m^2]

In [25]:
PvC=PC*Bolt_C/sum_C # [N/m^2]

In [26]:
print(PvSi)

[[2.15152111e+21 2.42631874e-02 4.77945821e-03 1.68533325e-05]
 [5.68239882e-02 3.46338764e-02 1.54468558e-03 2.15152111e+21]
 [3.79326319e-02 7.53248092e-03 2.15152111e+21 2.15152111e+21]
 [2.74535171e-02 2.15152111e+21 2.15152111e+21 2.15152111e+21]]


In [27]:
print(PvC)

[[1.02099958e+35 8.64100533e+08 2.64696859e+07 1.62815745e+06]
 [2.71869761e+04 7.39042284e+02 2.33070255e+01 1.02099958e+35]
 [4.03555074e-04 2.27653154e-04 1.02099958e+35 1.02099958e+35]
 [2.30545742e-05 1.02099958e+35 1.02099958e+35 1.02099958e+35]]


In [28]:
EvSi=PvSi*math.sqrt(mSi/(NA*2.0*pi*kbJ*Temp)) # [kg/(m^2 s)] # to be checked shold be [at/s]

In [29]:
EvC=PvC*math.sqrt(mC/(NA*2.0*pi*kbJ*Temp))

In [30]:
print(EvSi)

[[4.06698005e+19 4.58642486e-04 9.03452031e-05 3.18575387e-07]
 [1.07413320e-03 6.54677677e-04 2.91989022e-05 4.06698005e+19]
 [7.17033435e-04 1.42385076e-04 4.06698005e+19 4.06698005e+19]
 [5.18948690e-04 4.06698005e+19 4.06698005e+19 4.06698005e+19]]


In [31]:
print(EvC)

[[1.26346612e+33 1.06930675e+07 3.27556956e+05 2.01481158e+04]
 [3.36433275e+02 9.14549727e+00 2.88419679e-01 1.26346612e+33]
 [4.99391159e-06 2.81716127e-06 1.26346612e+33 1.26346612e+33]
 [2.85295646e-07 1.26346612e+33 1.26346612e+33 1.26346612e+33]]


In [32]:
PtransE_Si=EvSi
PtransE_C=EvC

## Deposition Frequencies

We calibrated the deposition frequencies from the experimental inverted pyramid growth along the [001] direction with marker layers.

In [33]:
PtransD=np.zeros(6)

In [34]:
fact_D = [1.10546, 200.0, 0.276365, 2.5, 44.2184, 400.0]

In [35]:
PtransD[0]=fact_D[0]*PtransE_Si[0,2] # Si coord 1, old 1
PtransD[1]=fact_D[1]*PtransE_C[2,0] # C  coord 1, old 1
PtransD[2]=fact_D[2]*PtransE_Si[0,2] # Si coord 2, old 2
PtransD[3]=fact_D[3]*PtransE_C[2,0]  # C  coord 2, old 2
PtransD[4]=fact_D[4]*PtransE_Si[0,2] # Si coord 3, old 20
PtransD[5]=fact_D[5]*PtransE_C[2,0] # C  coord 3, old 20

In [36]:
print(PtransD)

[9.98730082e-05 9.98782319e-04 2.49682521e-05 1.24847790e-05
 3.99492033e-03 1.99756464e-03]


## Generation of the kinetic Monte Carlo input file

In [38]:
filename="start.dat"

In [39]:
def print_file_input(filename,PtransE_Si,PtransE_C,PtransD,PtransZig,Simulation,Sedd_box):
    file = open(filename,"w") 

    file.write(str(PtransE_Si[0,1])+" ! PtransE_Si[0,1]" + "\n") 
    file.write(str(PtransE_Si[1,0])+" ! PtransE_Si[1,0]" + "\n")
    file.write(str(PtransE_Si[1,1])+" ! PtransE_Si[1,1]" + "\n") 
    file.write(str(PtransE_Si[0,2])+" ! PtransE_Si[0,2]" + "\n") 
    file.write(str(PtransE_Si[2,0])+" ! PtransE_Si[2,0]" + "\n") 
    file.write(str(PtransE_Si[2,1])+" ! PtransE_Si[2,1]" + "\n") 
    file.write(str(PtransE_Si[1,2])+" ! PtransE_Si[1,2]" + "\n") 
    file.write(str(PtransE_Si[0,3])+" ! PtransE_Si[0,3]" + "\n") 
    file.write(str(PtransE_Si[3,0])+" ! PtransE_Si[3,0]" + "\n") 

    file.write(str(PtransE_C[0,1])+" ! PtransE_C[0,1]" + "\n") 
    file.write(str(PtransE_C[1,0])+" ! PtransE_C[1,0]" + "\n")
    file.write(str(PtransE_C[1,1])+" ! PtransE_C[1,1]" + "\n") 
    file.write(str(PtransE_C[0,2])+" ! PtransE_C[0,2]" + "\n") 
    file.write(str(PtransE_C[2,0])+" ! PtransE_C[2,0]" + "\n") 
    file.write(str(PtransE_C[2,1])+" ! PtransE_C[2,1]" + "\n") 
    file.write(str(PtransE_C[1,2])+" ! PtransE_C[1,2]" + "\n") 
    file.write(str(PtransE_C[0,3])+" ! PtransE_C[0,3]" + "\n") 
    file.write(str(PtransE_C[3,0])+" ! PtransE_C[3,0]" + "\n") 

    file.write(str(PtransD[0])+" ! PtransD[1]" + "\n") 
    file.write(str(PtransD[1])+" ! PtransD[2]" + "\n") 
    file.write(str(PtransD[2])+" ! PtransD[3]" + "\n") 
    file.write(str(PtransD[3])+" ! PtransD[4]" + "\n") 
    file.write(str(PtransD[4])+" ! PtransD[5]" + "\n") 
    file.write(str(PtransD[5])+" ! PtransD[6]" + "\n") 

    file.write(str(PtransZig)+" ! PtransZig" + "\n")

    file.write(Simulation+" ! Initstat: S Sphere, C Parellelepipid, F Flat (100) surface, A APB, I inverted pyramid, D inverted pyramid of C, Z inverted pyramid of Si, J inverted pyramid with APB " +  "\n")

    Sys_size=str(Seed_box[0])+' '+str(Seed_box[1])+' '+str(Seed_box[2])
    file.write(Sys_size+" ! Len1 Len2 Len3" + "\n")
    
#    file.write("100000  ! OutMolMol -> output frequency" + "\n")
#    file.write("100000000 ! IterMax -> Max number of iterations" + "\n")
    
    file.write("100000  ! OutMolMol -> output frequency" + "\n")
    file.write("5000000 ! IterMax -> Max number of iterations" + "\n")

    
    file.write("10000   ! exit_zeta strategy after lenz=500 " + "\n")
    file.write("R   ! R = production run; T = run test with a fixed sequence of random numbers " + "\n")
    
    file.close() 
    return "printing done"

In [40]:
print_file_input('start-3C-SiC-APR-2020.dat',PtransE_Si,PtransE_C,PtransD,PtransZig,Simulation,Seed_box)

'printing done'